In [10]:
from E_coefficients_model import NirbModule
import numpy as np
import torch
import random 
import plotly.express as px
from pathlib import Path
from helpers import load_pint_data, inverse_min_max_scaler
import pyvista as pv

In [11]:
import sys
sys.path.append(str(Path().cwd().parent.parent))
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.helper import calculate_normal

In [12]:
PARAMETER_SPACE = "01"
ACCURACY = 1e-3
ROOT = Path.cwd().parent / "Snapshots" / PARAMETER_SPACE
ROOT.exists()

True

### Load checkpoints from different trainings

In [13]:
chk_pt_paths = [path for path in (ROOT / "nn_logs").rglob("*.ckpt")]
chk_pt_paths = sorted(chk_pt_paths, key = lambda p: p.parent.parent.stem)

for idx, path in enumerate(chk_pt_paths):
    print(f"{idx:02d} ({path.parent.parent.stem}): {path.name}")

00 (version_14): epoch=19999-step=60000.ckpt
01 (version_15): epoch=19999-step=60000.ckpt
02 (version_16): epoch=39999-step=120000.ckpt
03 (version_20): epoch=99999-step=300000.ckpt
04 (version_22): epoch=63962-step=191889.ckpt
05 (version_23): epoch=41471-step=82944.ckpt
06 (version_24): epoch=199999-step=400000.ckpt
07 (version_25): epoch=299999-step=600000.ckpt
08 (version_26): epoch=52520-step=105042.ckpt
09 (version_27): epoch=99999-step=200000.ckpt
10 (version_28): epoch=249999-step=500000.ckpt
11 (version_30): epoch=236356-step=236357.ckpt
12 (version_33): epoch=7174-step=7175.ckpt
13 (version_34): epoch=59529-step=59530.ckpt
14 (version_35): epoch=199999-step=200000.ckpt
15 (version_36): epoch=160489-step=160490.ckpt
16 (version_37): epoch=199999-step=200000.ckpt
17 (version_38): epoch=99999-step=100000.ckpt
18 (version_39): epoch=99999-step=100000.ckpt
19 (version_40): epoch=174756-step=174757.ckpt
20 (version_41): epoch=199999-step=200000.ckpt
21 (version_42): epoch=199999-st

In [14]:
# ckpt = torch.load(chk_pt_paths[10], map_location="cpu")
# for key in ckpt['state_dict'].keys():
#     print(key)
# for k, v in ckpt['state_dict'].items():
#     print(k, v.shape)
# print(ckpt.get("hyper_parameters", {}))
# print(ckpt.get("hyper_parameters", {}))

### Load checkpoint

In [15]:
# Select checkpoint
chk_pt_path = chk_pt_paths[20]

In [16]:
trained_model = NirbModule.load_from_checkpoint(chk_pt_path)
trained_model = trained_model.to('cpu')
trained_model.eval()

/Users/thomassimader/Documents/NIRB/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'activation' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['activation'])`.


NirbModule(
  (loss): MSELoss()
  (activation): Sigmoid()
  (model): NIRB_NN(
    (layers): Sequential(
      (0): Linear(in_features=2, out_features=6, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=6, out_features=44, bias=True)
      (3): Sigmoid()
      (4): Linear(in_features=44, out_features=96, bias=True)
      (5): Sigmoid()
      (6): Linear(in_features=96, out_features=80, bias=True)
      (7): Sigmoid()
      (8): Linear(in_features=80, out_features=34, bias=True)
      (9): Sigmoid()
      (10): Linear(in_features=34, out_features=7, bias=True)
    )
  )
  (msa_metric): MeanAbsoluteError()
)

In [17]:
def mse(predictions, targets):
    return np.mean(np.square(predictions - targets))

In [19]:
basis_functions             = np.load(ROOT / "BasisFunctions" / f"basis_fts_matrix_{ACCURACY:.1e}.npy")
training_snapshots_npy      = np.load(ROOT / "Exports" / "Training_temperatures.npy")
training_parameters     = load_pint_data(ROOT / "training_samples.csv", is_numpy = True)
min_max                     = np.load(ROOT / "BasisFunctions" / "min_max.npy")


test_snapshots_npy            = np.load(ROOT / "Test" / "Test_temperatures.npy")
test_parameters               = load_pint_data(ROOT / "test_samples.csv", is_numpy = True)
test_parameters_pint           = load_pint_data(ROOT / "test_samples.csv")



In [20]:
# Prepare data
training_snapshots  = training_snapshots_npy[:, -1, :] # last time step
test_snapshots      = test_snapshots_npy[:, -1, :]

In [21]:
print(test_snapshots.shape)
print(training_snapshots.shape)

(20, 20113)
(100, 20113)


In [22]:
# Calculate mean/var from training snapshots
mean = np.mean(training_parameters, axis=0)
var = np.var(training_parameters, axis=0)
# Standardization
training_parameters_scaled    = (training_parameters - mean)/np.sqrt(var)
test_parameters_scaled        = (test_parameters - mean)/np.sqrt(var)
predictions_train = trained_model(torch.from_numpy(training_parameters_scaled.astype(np.float32))) 
predictions_test  = trained_model(torch.from_numpy(test_parameters_scaled.astype(np.float32)))
full_solution_train = np.matmul(predictions_train.detach().numpy(), basis_functions)
full_solution_test = np.matmul(predictions_test.detach().numpy(), basis_functions)
mse_train = mse(full_solution_train, training_snapshots)
mse_test = mse(full_solution_test, test_snapshots )
# Variance of targets
# var_val = np.var(test_snapshots, axis=0)
var_train = np.var(training_snapshots, axis=0)

# R² and Q²
R2 = 1 - (np.sum((full_solution_train-training_snapshots)**2) / np.sum((training_snapshots-np.mean(training_snapshots))**2))
Q2 =  1 - (np.sum((full_solution_test-test_snapshots)**2) / np.sum((test_snapshots-np.mean(test_snapshots))**2))

# Aggregated if needed
toShowR2 = np.average(R2)
toReturnQ2 = np.average(Q2)
print(toShowR2)
print(toReturnQ2)
print(f"{np.average(mse_train):.3e}")
print(f"{np.average(mse_test):.3e}")

-66.80901464726716
-75.817097963175
1.385e+05
1.318e+05


### Prepare Plot

In [23]:
comsol_data = COMSOL_VTU("/Users/thomassimader/Documents/NIRB/Snapshots/01/Training/Training_000.vtu")
normal = calculate_normal(60, 90)

In [24]:
N = 5
random.seed(3123)
samples = random.sample(range(len(test_snapshots)), N)
colors = px.colors.sample_colorscale("thermal", [n/(N -1) for n in range(N)])

### Plot training vs original

In [ ]:
# Initialize the figure
n_cols = 2  # number of columns 
plotter = pv.Plotter(shape=(N, n_cols), window_size=(1300, 1200))
counter = 0
differences = np.zeros((N, len(comsol_data.mesh.points)))
# Loop through the samples and plot
for i in range(N):
    sample_idx = samples[i]
    plotter.subplot(i, 0)
    counter += 1
    field_name = f"Test Sample {sample_idx}"
    param_string =  "\n".join([f"{col} = {para.magnitude:.2e} {para.units:~P}" for col, para in test_parameters_pint.loc[sample_idx].items()])

    
    # Predict 
    param = test_parameters_scaled[sample_idx]
    param_t = torch.from_numpy(param.astype(np.float32)).view(1, -1)
    res = trained_model(param_t)
    res_np = res.detach().numpy()
    my_sol = np.dot(res_np.flatten(), basis_functions)
    my_sol = inverse_min_max_scaler(my_sol, min_max[0], min_max[1])
    
    comsol_data.mesh.point_data[field_name] = my_sol
    clipped = comsol_data.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
    
    plotter.add_mesh(clipped, scalars=field_name,
                        cmap='jet',
                        scalar_bar_args={'title': f'Temperature [K] ({counter})',
                                        'label_font_size': 10,
                                        'title_font_size': 8,})
    plotter.add_text(f"NIRB (Sample {sample_idx:03d})",
                     font_size=13)
    plotter.add_text(param_string,
                    position="left_edge",
                    font_size=9,)
    plotter.add_axes(line_width=1.)
    plotter.add_bounding_box()
    plotter.show_grid(
        font_size=6,
        n_xlabels=3,  # number of labels (ticks) on x-axis
        n_ylabels=3,  # number of labels (ticks) on y-axis
        n_zlabels=3,  # number of labels (ticks) on z-axis
        color='gray',
        xtitle='',
        ytitle='',
        ztitle='',
                        )
    plotter.subplot(i, 1)
    counter += 1
    comsol_data.mesh.point_data[f"Test{sample_idx}"] = test_snapshots[sample_idx, :]
    clipped = comsol_data.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
    plotter.add_mesh(clipped, scalars=f"Test{sample_idx}",
                    cmap='jet',
                    scalar_bar_args={'title': f'Temperature [K] ({counter})',
                                    'label_font_size': 10,
                                    'title_font_size': 8,})
    plotter.add_text(f"Original (Sample {sample_idx:03d})",
                     font_size=13)
    plotter.add_text(param_string,
                     position="left_edge",
                     font_size=9,)
    plotter.add_axes(line_width=1.)
    plotter.add_bounding_box()
    plotter.show_grid(
        font_size=6,
        n_xlabels=3,  # number of labels (ticks) on x-axis
        n_ylabels=3,  # number of labels (ticks) on y-axis
        n_zlabels=3,  # number of labels (ticks) on z-axis
        color='gray',
        xtitle='',
        ytitle='',
        ztitle='',
                        )
    
    
    differences[i, :] = test_snapshots[sample_idx, :] - my_sol
    
plotter.show(screenshot=ROOT / f"ComparisonNirbOriginal_PS{PARAMETER_SPACE}_{chk_pt_path.parents[1].stem}.png")


### Difference Plots

In [ ]:
# Initialize the figure
plotter = pv.Plotter(shape=(N, 1), window_size=(1300, 1200))
counter = 0
# Loop through the samples and plot
for i in range(N):
    sample_idx = samples[i]
    plotter.subplot(i, 0)
    counter += 1
    param_string =  "\n".join([f"{col} = {para.magnitude:.2e} {para.units:~P}" for col, para in test_parameters_pint.loc[sample_idx].items()])

    field_name = f"Difference (Reduced vs Full Solution) Sample {sample_idx}"
    
    comsol_data.mesh.point_data['diff'] = differences[i, :]
    clipped = comsol_data.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
    
    plotter.add_mesh(clipped, scalars='diff',
                        cmap='jet',
                        scalar_bar_args={'title': f'Diff Temperature [K] ({counter})',
                                        'label_font_size': 10,
                                        'title_font_size': 8,})
    plotter.add_text(f"Difference (Sample {sample_idx:03d})",
                     font_size=13)
    plotter.add_text(param_string,
                    position="left_edge",
                    font_size=10,)
    plotter.add_axes(line_width=1.)
    plotter.add_bounding_box()
    plotter.show_grid(
        font_size=6,
        n_xlabels=3,  # number of labels (ticks) on x-axis
        n_ylabels=3,  # number of labels (ticks) on y-axis
        n_zlabels=3,  # number of labels (ticks) on z-axis
        color='gray',
        xtitle='',
        ytitle='',
        ztitle='',
                        )

    
plotter.show(screenshot=ROOT / f"ComparisonNirbOriginal_Diff_PS{PARAMETER_SPACE}_{chk_pt_path.parents[1].stem}.png")


Widget(value='<iframe src="http://localhost:55424/index.html?ui=P_0x38d31d880_1&reconnect=auto" class="pyvista…